In [1]:
%matplotlib inline

import pandas as pd, numpy as np, seaborn as sns
import matplotlib.pyplot as plt

from qiime.parse import parse_mapping_file
from qiime.format import format_mapping_file
from skbio.io.util import open_file
from scipy.stats import pearsonr, spearmanr

def load_mf(fn):
    with open_file(fn, 'U') as f:
        mapping_data, header, _ = parse_mapping_file(f)
        _mapping_file = pd.DataFrame(mapping_data, columns=header)
        _mapping_file.set_index('SampleID', inplace=True)
    return _mapping_file

def write_mf(f, _df):
    with open_file(f, 'w') as fp:
        lines = format_mapping_file(['SampleID'] + _df.columns.tolist(),
                                    list(_df.itertuples()))
        fp.write(lines+'\n')

Download the files from Embriette:

In [8]:
!rm -rf combined-gevers-suchdolski/picrust

In [9]:
!mkdir -p combined-gevers-suchdolski/picrust/
!cp -rf /Users/yoshikivazquezbaeza/Dropbox/16s-fecal-only/beta_div_even430973_full_table/* \
combined-gevers-suchdolski/picrust/

In [10]:
ls combined-gevers-suchdolski/picrust

binary_jaccard_dm.txt
binary_jaccard_emperor_pcoa_plot/
binary_jaccard_pc.txt
log_20160803133516.txt
predicted_metagenome_cat_by_function_l3_filtered_even4303973.biom


In [11]:
mf = load_mf('combined-gevers-suchdolski/mapping-file.standardized.alpha.amended.fecal-only.txt')
mf['COMBINED_HOST_AND_IBD'] = (mf.HOST_COMMON_NAME.replace(to_replace='domestic dog', value='dog')
                               + ' ' + mf.IBD_STATUS.replace('Healthy', 'non-IBD'))
write_mf('combined-gevers-suchdolski/mapping-file.standardized.alpha.amended.fecal-only.combined-columns.txt', mf)

In [12]:
!make_emperor.py \
-i combined-gevers-suchdolski/picrust/binary_jaccard_pc.txt \
-m combined-gevers-suchdolski/mapping-file.standardized.alpha.amended.fecal-only.combined-columns.txt \
-o combined-gevers-suchdolski/picrust/binary_jaccard_emperor_pcoa_plot/ \

In [13]:
!compare_categories.py --method permanova -i combined-gevers-suchdolski/picrust/binary_jaccard_dm.txt \
-m combined-gevers-suchdolski/mapping-file.standardized.alpha.amended.fecal-only.combined-columns.txt \
-c IBD_STATUS -o combined-gevers-suchdolski/picrust/permanova-res-IBD_STATUS

In [14]:
ls combined-gevers-suchdolski/picrust/permanova-res-IBD_STATUS/

permanova_results.txt


In [15]:
pd.read_csv('combined-gevers-suchdolski/picrust/permanova-res-IBD_STATUS/permanova_results.txt', sep='\t')

,method name,PERMANOVA
0,test statistic name,pseudo-F
1,sample size,628
2,number of groups,2
3,test statistic,14.617123152059563
4,p-value,0.001
5,number of permutations,999


In [16]:
!compare_categories.py --method permanova -i combined-gevers-suchdolski/picrust/binary_jaccard_dm.txt \
-m combined-gevers-suchdolski/mapping-file.standardized.alpha.amended.fecal-only.combined-columns.txt \
-c HOST_COMMON_NAME -o combined-gevers-suchdolski/picrust/permanova-res-HOST_COMMON_NAME

pd.read_csv('combined-gevers-suchdolski/picrust/permanova-res-HOST_COMMON_NAME/permanova_results.txt', sep='\t')

,method name,PERMANOVA
0,test statistic name,pseudo-F
1,sample size,628
2,number of groups,2
3,test statistic,52.590980434970973
4,p-value,0.001
5,number of permutations,999
